In [1]:
from fastai import *
from fastai.tabular import *
import pandas as pd
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F
import collections

In [2]:
colab = False

In [3]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    %cd "/content/drive/My Drive/automatic-asset-classification"
    %ls "/content/drive/My Drive/automatic-asset-classification"
    image_path = "/content/drive/My Drive/automatic-asset-classification/data/final_dataset"
    
if not colab:
    image_path = '/Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/'\
                 'final_dataset/final'

In [16]:
size = 224
batchsize = 32
tfms = get_transforms(do_flip = False)
src = (ImageImageList.from_folder(image_path, convert_mode = 'L').split_by_rand_pct(seed=2).label_from_func(lambda 
                                                                                                            x: x))
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(do_y = False))

In [35]:
src = (ImageImageList.from_folder(image_path, convert_mode = 'L').split_by_rand_pct(seed=2).\
       label_from_func(lambda x: x, convert_mode = 'L'))

In [36]:
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(do_y = False))

In [37]:
class LinearAutoEncoder(nn.Module):

    def __init__(self):
        super(LinearAutoEncoder, self).__init__()
        
        self.layer_outs = [int(224*224//(6.125**i)) for i in range(6)];
        self.enc = [];
        self.dec = [];

        for i in range(len(self.layer_outs) - 1):
            self.enc += [
                nn.Linear(self.layer_outs[i], self.layer_outs[i+1]),
                nn.BatchNorm1d(self.layer_outs[i+1], eps=1e-05, momentum=0.1, affine=True, 
                               track_running_stats=True),
                nn.Tanh()
            ]
            
        self.layer_outs.reverse()
        for i in range(len(self.layer_outs) - 1):
            self.dec += [
                nn.Linear(self.layer_outs[i], self.layer_outs[i+1]),
                nn.BatchNorm1d(self.layer_outs[i+1], eps=1e-05, momentum=0.1, affine=True, 
                               track_running_stats=True),
                nn.Tanh()
            ]

        self.encoder = nn.Sequential(*self.enc)
        self.decoder = nn.Sequential(*self.dec)

        
    def encode(self, x): return self.encoder(x)
    
    def decode(self, x): return self.decoder(x)

    def forward(self, x):
        x = x.view(-1, 224**2)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        decoded = decoded.reshape([-1,1,224,224])
        return decoded

In [38]:
autoencoder = LinearAutoEncoder();

In [39]:
learn = Learner(data, autoencoder, loss_func=F.mse_loss);

In [40]:
from torchsummary import summary
summary(autoencoder, (1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 8192]     411,049,984
       BatchNorm1d-2                 [-1, 8192]          16,384
              Tanh-3                 [-1, 8192]               0
            Linear-4                 [-1, 1337]      10,954,041
       BatchNorm1d-5                 [-1, 1337]           2,674
              Tanh-6                 [-1, 1337]               0
            Linear-7                  [-1, 218]         291,684
       BatchNorm1d-8                  [-1, 218]             436
              Tanh-9                  [-1, 218]               0
           Linear-10                   [-1, 35]           7,665
      BatchNorm1d-11                   [-1, 35]              70
             Tanh-12                   [-1, 35]               0
           Linear-13                    [-1, 5]             180
      BatchNorm1d-14                   

In [ ]:
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,time


In [8]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False);
        self.batch1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True);
        
        self.up1 = nn.Upsample(scale_factor = 2, mode = 'bilinear');
        self.conv2 = nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False);
        
        self.relu = nn.ReLU(inplace = True)
        
    def encoder(self, x):
        return nn.Sequential(self.conv1, self.batch1, self.relu)(x)
    
    def decoder(self, x):
        return nn.Sequential(self.up1,self.conv2, self.relu)(x)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [62]:
autoencoder2 = AutoEncoder();
autoencoder2

AutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (up1): Upsample(scale_factor=2.0, mode=bilinear)
  (conv2): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (relu): ReLU(inplace=True)
)

In [63]:
learn2 = Learner(data, autoencoder2, loss_func=F.mse_loss);

In [66]:
learn2.model

AutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (up1): Upsample(scale_factor=2.0, mode=bilinear)
  (conv2): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (relu): ReLU(inplace=True)
)